In [ ]:
#### import global modules
import os
import sys
import pandas as pd
import numpy as np
from pathlib import Path
from yaml import safe_load
import google.oauth2.credentials
from google.cloud import bigquery
import gc

# Set global vars
pth_project = Path(os.getcwd().split('notebooks')[0])
pth_data = pth_project / 'data'
pth_queries = pth_project / 'core' / 'queries'
pth_creds = pth_project / 'conf' / 'local' / 'project_config.yaml'
sys.path.insert(0, str(pth_project))
d_project_config = safe_load(pth_creds.open())
# d_params = safe_load((pth_project / 'core' / 'parameters' / 'common.yaml').open())['data_extract']

# import local modules
from core.utils.gcp import connect_bq_services
# from core.etl.extract import extract_bq_data, extract_pr_codes, format_conv_df, filter_convs

# Connect to google services
bq_client = connect_bq_services(d_project_config['gcp-project-name'])
pd.options.display.max_rows = 100

In [ ]:
def extract_bq_data(bq_client, sql=None, pth_query=None):
    if sql is not None:
        df = bq_client.query(sql).to_dataframe()
    elif pth_query is not None:
        sql = pth_query.read_text()
        df = bq_client.query(sql).to_dataframe()
    else:
        raise ValueError('`sql` or `pth_query` should be set')  
    return df

In [ ]:
Query='''


with ADC_data as 

(

SELECT *
    

   FROM
    `divgpras-pr-579355.ADC_Feature_Datastore.ADC_Master_Data`
  WHERE
    Month_Snapshot='2022-12-01'
    AND dealer_name='TELUS Communications Inc.' 
    and Segment='Heavy_User'
    
)

,Telus_customers as
(
select cust_bus_cust_id,pi_cntrct_start_ts as contract_start_date,pi_cntrct_end_ts as contract_end_date
from `cio-datahub-enterprise-pr-183a.ent_cust_cust.bq_prod_instnc_snpsht` 
WHERE DATE(prod_instnc_ts) = '2022-12-31'
and pi_prod_instnc_typ_cd ='SMHM' #Serice type
and bus_prod_instnc_src_id = 1001 #BANs that are for home services
and pi_prod_instnc_stat_cd in ('A')
and  consldt_cust_typ_cd = 'R'
order by cust_bus_cust_id

)

select * from ADC_data a
inner join Telus_customers b
on a.dealer_customer_id=b.cust_bus_cust_id


'''

In [ ]:
DF=extract_bq_data(bq_client, sql=Query)

In [ ]:
DF['account_type_name'].value_counts()

In [ ]:
DF.info(verbose=True)

In [ ]:
TC_columns= [col for col in DF.columns if 'TC_' in col]

In [ ]:
DF['TC_columns_total']=DF[TC_columns].apply(lambda x: x.sum(),axis=1)

In [ ]:
DF['TC_columns_total'].value_counts()

In [ ]:
DF_1=DF[DF.TC_columns_total==0]

In [ ]:
DF_1.info()

In [ ]:
DF_1.to_csv('Customer_review_sizing.csv',index=False)